In [1]:
import numpy as np
import math
from sklearn.cross_validation import KFold
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
%matplotlib inline
import math
from sklearn.metrics import accuracy_score, average_precision_score, precision_score,recall_score,roc_curve,precision_recall_curve, auc,f1_score, confusion_matrix
import operator
from scipy.optimize import fmin_bfgs

In [1]:
from sklearn.datasets import fetch_mldata
import numpy as np
import os
from pylab import *
mnist = fetch_mldata('MNIST original')
mnist.data.shape
mnist.target.shape
np.unique(mnist.target)
X_m, y_m = mnist.data / 255., mnist.target
X_train, X_test = X_m[:10000], X_m[10000:]
y_train, y_test = y_m[:10000], y_m[10000:]
X_train.shape
y_train.shape
size=len(y_train)

In [6]:
X_train_m= []
Y_train_m=[]
X_test_m= []
Y_test_m=[]
for i in range(y_test.shape[0]):
    if y_test[i] == 0.0:
        X_train_m.append(X_test[i])
        Y_train_m.append(-1);
    elif y_test[i] == 1.0:
        X_train_m.append(X_test[i])
        Y_train_m.append(1);
X_train_m=np.array(X_train_m)
Y_train_m=np.array(Y_train_m)
print X_train_m.shape
print Y_train_m.shape
print set(Y_train_m)
for i in range(y_train.shape[0]):
    if y_train[i] == 0.0:
        X_test_m.append(X_test[i])
        Y_test_m.append(-1);
    elif y_train[i] == 1.0:
        X_test_m.append(X_test[i])
        Y_test_m.append(1);
X_test_m=np.array(X_test_m[5500:6500])
Y_test_m=np.array(Y_test_m[5500:6500])
print X_test_m.shape
print Y_test_m.shape
print set(Y_test_m)

(4780L, 784L)
(4780L,)
set([1, -1])
(1000L, 784L)
(1000L,)
set([1, -1])


In [11]:
def read_input(filename):
    input_data=[];
    f=open(filename);
    #f.readline();
    #f.readline();
    for line in f:
        input_data.append(line.rstrip('\n').split(','));
    return input_data;

def create_feature_matrix(input_data):
    x_list=[a[0:len(input_data[0])-1] for a in input_data];
    x=np.array([[1]+a[0:len(input_data[0])-1] for a in input_data]);
    #x=np.transpose(x)
    x= x.astype(np.float);
    return x[0:100]

def create_y_matrix(input_data):
    y_list=[a[len(input_data[0])-1] for a in input_data];
    y_list_1=[];
    for i in y_list:
        if(i=='Iris-setosa'):
            y_list_1.append(-1);
        elif(i=='Iris-versicolor'):
            y_list_1.append(1);
        elif(i=='Iris-virginica'):
            y_list_1.append(2);
        
            
    y=np.array(y_list_1);
    #y=np.transpose(y)
    y= y.astype(np.float);
    return y[0:100]

In [12]:
#input_data=read_input('winequality-red.csv')
input_data=read_input('iris.dat')
X=create_feature_matrix(input_data)
Y=create_y_matrix(input_data)
print X.shape
print Y.shape

(100L, 5L)
(100L,)


In [4]:
def h_theta( x):
    #print x.shape
    #exp_theta=math.exp(np.dot(x,theta))
    #den=1+exp_theta;
    h_theta_den = 1.0 + np.exp(-x)
    return 1.0/h_theta_den;

In [5]:
def l_theta(x,y, theta,c=0 ):
    """m=y.shape[0];
    print theta.shape
    print x.shape
    print (np.dot(x,np.transpose(np.matrix(theta)))).shape
    h_t=h_theta(np.transpose(np.matrix(theta))*x);
    l_theta=np.sum((y*log(h_t))-((1-y)-log(1-h_t)));
    l_theta=1.*l_theta/m;
    return l_theta"""
    h_t=h_theta(y*np.dot(x, theta));
    log_theta=np.sum(np.log(h_t));
    log_theta=log_theta-c/2 * np.dot(theta,theta)
    return log_theta;

In [6]:
def gradient(x,y,theta, c=0.1):
    result=np.zeros(len(theta))
    for i in range(x.shape[0]):
        result=result+y[i]*x[i]*h_theta(-y[i]*np.dot(x[i],theta))
    result=result-c*theta;
    return result;

In [7]:
def predict(x_test, y_test, theta):
    predicted_y=[];
    for i in range(x_test.shape[0]):
        discriminant=h_theta(np.dot(theta,x_test[i]));
        #print discriminant
        if(discriminant>0.5):
            predicted_y.append(1);
        else:
            predicted_y.append(-1);
    return predicted_y;

In [8]:
def calc_theta(x,y,c =0):
    def optimize_theta(theta):
        return -l_theta(x,y,theta,c)
    
    def optimize_grad(theta):
        return -gradient(x,y,theta,c)
    
    initial_theta=np.zeros(x.shape[1]);
    
    return fmin_bfgs(optimize_theta, initial_theta, optimize_grad)

In [9]:
def do_cross_validation(X, y,  classarray,n_folds, poly,poly_degree, verbose=False):
    
    accuracies = []
    precisions =[]
    recalls=[]
    f1_scores=[]
    y_p=[]
    i=1;
    #X1, Y1= X[0:100],y[0:100]
    X1, Y1= X,y
    if(poly==True):
        poly = PolynomialFeatures(poly_degree);
        X1=poly.fit_transform(X1);
    cv = KFold(len(Y1), n_folds, shuffle=True, random_state=5)
    
    for train_idx, test_idx in cv:
        theta=calc_theta(X1[train_idx],Y1[train_idx]);
        print theta
        predicted_y=predict(X1[test_idx], Y1[test_idx], theta);
        #print Y1[test_idx];
        #print predicted_y;
        acc = accuracy_score(Y1[test_idx], predicted_y)
        c_matrix= confusion_matrix(Y1[test_idx], predicted_y)
        precision=precision_score(Y1[test_idx], predicted_y, average="binary")
        recall=recall_score(Y1[test_idx], predicted_y, average="binary")
        f_score=f1_score(Y1[test_idx], predicted_y, average="binary")
        if(verbose == True):
            print('fold %d accuracy=%.4f, precision=%.4f, recall=%.4f, F-measure%.4f, confusion matrix=' % (i, acc, precision, recall, f_score));
            print c_matrix
        i=i+1;
        accuracies.append(acc)
        precisions.append(precision)
        recalls.append(recall)
        f1_scores.append(f_score)
        prredicted_y=[]
    avg=np.max(accuracies);
    avg_precision=np.max(precisions)
    avg_recall=np.max(recalls)
    avg_f1_score=np.max(f1_scores)
    return avg, avg_precision, avg_recall, avg_f1_score;

## Cross Validation

In [13]:
do_cross_validation(X, Y,[1,2],10, False, 0, verbose=True)

Optimization terminated successfully.
         Current function value: 0.000001
         Iterations: 28
         Function evaluations: 32
         Gradient evaluations: 32
[ -4.08442601  -1.67302281 -10.74307778  11.31687977  17.58654963]
fold 1 accuracy=1.0000, precision=1.0000, recall=1.0000, F-measure1.0000, confusion matrix=
[[8 0]
 [0 2]]
Optimization terminated successfully.
         Current function value: 0.000001
         Iterations: 28
         Function evaluations: 32
         Gradient evaluations: 32
[ -7.18219093  -1.74894466 -10.76804986  12.54392136  17.26906789]
fold 2 accuracy=1.0000, precision=1.0000, recall=1.0000, F-measure1.0000, confusion matrix=
[[5 0]
 [0 5]]
Optimization terminated successfully.
         Current function value: 0.000001
         Iterations: 28
         Function evaluations: 32
         Gradient evaluations: 32
[ -5.84180904  -2.91048778 -10.74111593  14.87126728  15.08661855]
fold 3 accuracy=1.0000, precision=1.0000, recall=1.0000, F-measure1.0

(1.0, 1.0, 1.0, 1.0)

##Overall Performance 

In [28]:
X1, Y1= X[0:100],Y[0:100]
theta=calc_theta(X1,Y1);
print theta
predicted_y=predict(X1, Y1, theta);
print "Accuracy: %f" %accuracy_score(Y1, predicted_y)
print "Precision: %f" %precision_score(Y1, predicted_y, average="binary")
print "Recall: %f" %recall_score(Y1, predicted_y, average="binary")
print "F1 Scpre: %f" %f1_score(Y1, predicted_y, average="binary")
print "Confusion Matrix:"
print confusion_matrix(Y1, predicted_y)

Optimization terminated successfully.
         Current function value: 0.000001
         Iterations: 28
         Function evaluations: 32
         Gradient evaluations: 32
[ -5.45390546  -1.72640044 -11.03607063  12.40692912  16.55051744]
Accuracy: 1.000000
Precision: 1.000000
Recall: 1.000000
F1 Scpre: 1.000000
Confusion Matrix:
[[50  0]
 [ 0 50]]


In [40]:
do_cross_validation(X_train_m, Y_train_m,[1,2],10, False, 0, verbose=True)

Optimization terminated successfully.
         Current function value: 0.000008
         Iterations: 119
         Function evaluations: 124
         Gradient evaluations: 124
[  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   8.99269947e+00   2.24817487e+01
   2.90940277e+00   0.00000000e+00   0.00000000e+00   0.00000000e+

C:\Users\Pooja\Anaconda\lib\site-packages\IPython\kernel\__main__.py:5: RuntimeWarning: overflow encountered in exp
C:\Users\Pooja\Anaconda\lib\site-packages\IPython\kernel\__main__.py:11: RuntimeWarning: divide by zero encountered in log


(0.99581589958159, 0.99730458221024254, 1.0, 0.99746192893401009)

In [53]:
X1, Y1= X_train_m, Y_train_m
theta=calc_theta(X1,Y1);
print theta
predicted_y=predict(X1, Y1, theta);
print "Accuracy: %f" %accuracy_score(Y1, predicted_y)
print "Precision: %f" %precision_score(Y1, predicted_y, average="binary")
print "Recall: %f" %recall_score(Y1, predicted_y, average="binary")
print "F1 Scpre: %f" %f1_score(Y1, predicted_y, average="binary")
print "Confusion Matrix:"
print confusion_matrix(Y1, predicted_y)

Optimization terminated successfully.
         Current function value: 0.000008
         Iterations: 92
         Function evaluations: 97
         Gradient evaluations: 97
[  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   2.88790452e+01   7.21976130e+01
   9.34322051e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00


C:\Users\Pooja\Anaconda\lib\site-packages\IPython\kernel\__main__.py:5: RuntimeWarning: overflow encountered in exp
C:\Users\Pooja\Anaconda\lib\site-packages\IPython\kernel\__main__.py:11: RuntimeWarning: divide by zero encountered in log


## Cross Validation for non linear input

In [29]:
do_cross_validation(X, Y,[1,2],10, True, 2, verbose=True)

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 33
         Function evaluations: 37
         Gradient evaluations: 37
[ -2.92087076  -2.92087076  -1.47967292 -11.56063913  20.75855115
   9.46526173  -2.92087076  -1.47967292 -11.56063913  20.75855115
   9.46526173  -1.33277219   4.41837006  -0.14712387   2.96768901
  -2.40288022  -0.46501759   3.13213578  -4.28544275  -5.07897809
  -4.18469065]
fold 1 accuracy=1.0000, precision=1.0000, recall=1.0000, F-measure1.0000, confusion matrix=
[[8 0]
 [0 2]]
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 35
         Function evaluations: 39
         Gradient evaluations: 39
[ -9.60673096  -9.60673096 -10.92205448 -27.61208567  39.32056452
  15.9113951   -9.60673096 -10.92205448 -27.61208567  39.32056452
  15.9113951   -0.7536284    9.02203771   5.53548687  -4.6178199
   1.77448084  -7.02566959  -8.47953122  -6.52243648  -2.2423077
  -0.781

C:\Users\Pooja\Anaconda\lib\site-packages\IPython\kernel\__main__.py:5: RuntimeWarning: overflow encountered in exp
C:\Users\Pooja\Anaconda\lib\site-packages\IPython\kernel\__main__.py:11: RuntimeWarning: divide by zero encountered in log


(1.0, 1.0, 1.0, 1.0)

## Overall performance for non-linear input

In [30]:
X1, Y1= X[0:100],Y[0:100]
poly = PolynomialFeatures(2);
X1=poly.fit_transform(X1);
theta=calc_theta(X1,Y1);
print theta
predicted_y=predict(X1, Y1, theta);
print "Accuracy: %f" %accuracy_score(Y1, predicted_y)
print "Precision: %f" %precision_score(Y1, predicted_y, average="binary")
print "Recall: %f" %recall_score(Y1, predicted_y, average="binary")
print "F1 Scpre: %f" %f1_score(Y1, predicted_y, average="binary")
print "Confusion Matrix:"
print confusion_matrix(Y1, predicted_y)

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 32
         Function evaluations: 36
         Gradient evaluations: 36
[ -3.2075114   -3.2075114   -3.1052183  -11.11058757  17.87403261
   7.85228485  -3.2075114   -3.1052183  -11.11058757  17.87403261
   7.85228485  -1.04978591   3.76090376   2.88489049   1.64431077
  -1.44829682   0.47494692   0.6250598   -6.3425588   -4.3439567
  -3.09739348]
Accuracy: 1.000000
Precision: 1.000000
Recall: 1.000000
F1 Scpre: 1.000000
Confusion Matrix:
[[50  0]
 [ 0 50]]


C:\Users\Pooja\Anaconda\lib\site-packages\IPython\kernel\__main__.py:5: RuntimeWarning: overflow encountered in exp
C:\Users\Pooja\Anaconda\lib\site-packages\IPython\kernel\__main__.py:11: RuntimeWarning: divide by zero encountered in log


In [ ]:
do_cross_validation(X_train_m, Y_train_m,[1,2],10, True, 2, verbose=True)